In [1]:
import pandas as pd
import numpy as np

In [2]:
#import excel files into pandas dataframes
team_2017_2018 = pd.read_excel('/Users/willhall/Desktop/python_projects/DraftKings/2017-2018_NBA_Box_Score_Team-Stats.xlsx')
player_2017_2018 = pd.read_excel('/Users/willhall/Desktop/python_projects/DraftKings/NBA-2017-2018-Player-BoxScore-Dataset.xlsx')
DFS_2017_2018 = pd.read_excel('/Users/willhall/Desktop/python_projects/DraftKings/NBA-2017-2018-DFS-Dataset.xlsx')

In [3]:
#rename colums so the column names are on continuous string
player_2017_2018 = player_2017_2018.rename(index=str, columns={'DATA SET':'DATASET','PLAYER FULL NAME':'PLAYER','OWN TEAM':'TEAM','OPP TEAM':'OPPONENT','VENUE (R/H)':'ROAD_HOME','MIN':'MINUTES'})

In [53]:
#rename 'TEAMS' to 'TEAM' to match other DFs and remove spaces in column names
team_2017_2018 = team_2017_2018.rename(index=str, columns={'TEAMS':'TEAM','REST DAYS':'REST_DAYS','OPENING SPREAD':'OPENING_SPREAD','OPENING TOTAL':'OPENING_TOTAL'})



In [54]:
#create list of all teams
teams = team_2017_2018.TEAM.unique()

In [55]:
#create new DataFrame for all of Boston's games
boston = pd.DataFrame(team_2017_2018.loc[lambda df: team_2017_2018.TEAM == 'Boston'])
#create new index
boston = boston.reset_index(drop=True)
#shift index up by 1 - so the index starts at 1
boston.index += 1
#Create Game# column to indicate the game of the season
boston['GAME#'] = boston.index
#reset index
boston.reset_index(drop=True,inplace=True)
#set higherarchical index on date and team
boston = boston.set_index(['DATE','TEAM'])


boston.head()

,,DATASET,VENUE,1Q,2Q,3Q,4Q,OT1,OT2,OT3,OT4,...,OPENING ODDS,OPENING_SPREAD,OPENING_TOTAL,MOVEMENTS,CLOSING ODDS,MONEYLINE,HALFTIME,BOX SCORE,ODDS,GAME#
DATE,TEAM,,,,,,,,,,,,,,,,,,,,,
10/17/2017,Boston,2017-2018 Regular Season,Road,19,19,33,28,NaN,NaN,NaN,NaN,...,214.5,4.0,214.5,215 / 215.5 / 215.5o15,216.0,166,110,NaN,NaN,1
10/18/2017,Boston,2017-2018 Regular Season,Home,26,27,27,20,NaN,NaN,NaN,NaN,...,-6.0,-6.0,208.5,-1.5 / -1.5 -05 / -1 -11,-1.0,-120,-2,NaN,NaN,2
10/20/2017,Boston,2017-2018 Regular Season,Road,24,22,23,33,NaN,NaN,NaN,NaN,...,-1.0,-1.0,216.5,217.5 / 216.5 / 217,216.5,125,112,NaN,NaN,3
10/24/2017,Boston,2017-2018 Regular Season,Home,30,24,23,33,NaN,NaN,NaN,NaN,...,-9.0,-9.0,210.0,-8 / -7.5 -05 / -7,-7.0,-280,+2 -15,NaN,NaN,4
10/26/2017,Boston,2017-2018 Regular Season,Road,28,15,29,24,NaN,NaN,NaN,NaN,...,207.0,3.5,207.0,207 / 206.5 / 207,206.5,163,106,NaN,NaN,5


In [56]:
player_2017_2018.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27833 entries, 0 to 27832
Data columns (total 23 columns):
DATASET      27833 non-null object
DATE         27833 non-null object
PLAYER       27833 non-null object
POSITION     27833 non-null object
TEAM         27833 non-null object
OPPONENT     27833 non-null object
ROAD_HOME    27833 non-null object
MINUTES      27833 non-null float64
FG           27833 non-null int64
FGA          27833 non-null int64
3P           27833 non-null int64
3PA          27833 non-null int64
FT           27833 non-null int64
FTA          27833 non-null int64
OR           27833 non-null int64
DR           27833 non-null int64
TOT          27833 non-null int64
A            27833 non-null int64
PF           27833 non-null int64
ST           27833 non-null int64
TO           27833 non-null int64
BL           27833 non-null int64
PTS          27833 non-null int64
dtypes: float64(1), int64(15), object(7)
memory usage: 5.1+ MB


In [57]:
#create DF of all the individual game stats for all boston players
boston_players = pd.DataFrame(player_2017_2018.loc[lambda df: player_2017_2018.TEAM == 'Boston'])
#reset index of boston_players DF
boston_players = boston_players.reset_index(drop=True)



In [58]:
# >>> df['newcolumn'] = df.apply(lambda x: function, axis=1)


In [59]:
boston_players.DATASET.count()

1078

In [60]:
boston_players.columns

Index(['DATASET', 'DATE', 'PLAYER', 'POSITION', 'TEAM', 'OPPONENT',
       'ROAD_HOME', 'MINUTES', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'OR',
       'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS'],
      dtype='object')

In [61]:
#create column of all zeros - this column will be the double-double bonus
boston_players['DT_Double'] = pd.Series(np.zeros(boston_players.DATASET.count()))

In [62]:
#This counts the number of categories the player had more than 10
categories = ['PTS', 'TOT', 'A', 'BL', 'ST']
for cat in categories:
    for i,x in enumerate(boston_players[cat]):
        if x >=10:
            boston_players['DT_Double'][i] += 1

/Users/willhall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [63]:
#This adjusts the 'DT_Double column to the appropriate multiplier
for i,x in enumerate(boston_players['DT_Double']):
    if x == 1 or x == 2:
        boston_players['DT_Double'][i] -= 1

/Users/willhall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [64]:
#create column of draftkings points
boston_players['DraftKings_PTS'] = boston_players['PTS'] + (boston_players['3P'] * 0.5) + (boston_players['TOT'] * 1.25) + (boston_players['A'] * 1.5) + (boston_players['ST'] * 2) + (boston_players['BL'] * 2) - (boston_players['TO'] * 0.5) + (boston_players['DT_Double'] * 1.5) 

In [65]:
# z['c'] = z.apply(lambda row: 0 if row['b'] in (0,1) else row['a'] / math.log(row['b']), axis=1)
# df['c']=df['b'].apply(lambda x: 0 if x ==0 else math.log(x))
# z['c'] = z.apply(lambda x: math.log(x.b) if x.b > 0 else 0, axis=1)
# sample['PR'] = sample['PR'].mask(sample['PR'] < 90, np.nan)


In [66]:
boston_players.head()

,DATASET,DATE,PLAYER,POSITION,TEAM,OPPONENT,ROAD_HOME,MINUTES,FG,FGA,...,DR,TOT,A,PF,ST,TO,BL,PTS,DT_Double,DraftKings_PTS
0,2017-2018 Regular Season,10/17/2017,Gordon Hayward,SF,Boston,Cleveland,R,5.25,1,2,...,1,1,0,1,0,0,0,2,0.0,3.25
1,2017-2018 Regular Season,10/17/2017,Jayson Tatum,SF,Boston,Cleveland,R,36.53,5,12,...,6,10,3,4,0,1,0,14,1.0,32.50
2,2017-2018 Regular Season,10/17/2017,Al Horford,C,Boston,Cleveland,R,32.12,2,7,...,7,7,5,2,0,0,1,9,0.0,27.25
3,2017-2018 Regular Season,10/17/2017,Jaylen Brown,SF,Boston,Cleveland,R,39.60,11,23,...,5,6,0,5,2,3,0,25,0.0,36.00
4,2017-2018 Regular Season,10/17/2017,Kyrie Irving,PG,Boston,Cleveland,R,39.35,8,17,...,2,4,10,4,3,2,0,22,1.0,50.50


In [67]:
DFS_2017_2018.head()

,DATASET,DATE,PLAYER,TEAM,OPPONENT,ROAD_HOME,MINUTES,USAGE,DK_POS,DK_SAL,DK_PTS
0,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Boston,Cleveland,R,5.25,16.473616,SF,6700,3.25
1,2017-2018 Regular Season,10/17/2017,Jayson Tatum,Boston,Cleveland,R,36.53,16.951635,SF/PF,3900,32.5
2,2017-2018 Regular Season,10/17/2017,Al Horford,Boston,Cleveland,R,32.12,13.570731,C,6000,27.25
3,2017-2018 Regular Season,10/17/2017,Jaylen Brown,Boston,Cleveland,R,39.60,29.352989,SG/SF,3700,36
4,2017-2018 Regular Season,10/17/2017,Kyrie Irving,Boston,Cleveland,R,39.35,21.846904,PG,8300,50.5


In [68]:
#Rename columns to names with one cintinuous string
DFS_2017_2018 = DFS_2017_2018.rename(index=str, columns={'VENUE\nR/H':'ROAD_HOME','USAGE \nRATE (%)':'USAGE','POSITION': 'DK_POS','Unnamed: 9':'FD_POS', 'Unnamed: 10':'YH_POS',
       'SALARY ($)':'DK_SAL', 'Unnamed: 12':'FD_SAL', 'Unnamed: 13':'YH_SAL', 'FANTASY POINTS SCORED':'DK_PTS',
       'Unnamed: 15':'FD_PTS', 'Unnamed: 16':'YH_PTS'})
#Drop Yahoo and FanDuels columns
DFS_2017_2018.drop(['YH_POS', 'YH_SAL','YH_PTS','FD_POS','FD_SAL','FD_PTS'], axis=1,inplace=True)
#Drop empty row
DFS_2017_2018.drop(DFS_2017_2018.index[0],inplace=True)
#Reset index
DFS_2017_2018 = DFS_2017_2018.reset_index(drop=True)

ValueError: labels ['YH_POS' 'YH_SAL' 'YH_PTS' 'FD_POS' 'FD_SAL' 'FD_PTS'] not contained in axis

In [69]:
#create DF of all the DFS stats for all boston players
DFS_boston_players = pd.DataFrame(DFS_2017_2018.loc[lambda df: DFS_2017_2018.TEAM == 'Boston'])
#reset index of DFS_boston_players DF
DFS_boston_players = DFS_boston_players.reset_index(drop=True)

In [70]:
DFS_boston_players.head()

,DATASET,DATE,PLAYER,TEAM,OPPONENT,ROAD_HOME,MINUTES,USAGE,DK_POS,DK_SAL,DK_PTS
0,2017-2018 Regular Season,10/17/2017,Gordon Hayward,Boston,Cleveland,R,5.25,16.473616,SF,6700,3.25
1,2017-2018 Regular Season,10/17/2017,Jayson Tatum,Boston,Cleveland,R,36.53,16.951635,SF/PF,3900,32.5
2,2017-2018 Regular Season,10/17/2017,Al Horford,Boston,Cleveland,R,32.12,13.570731,C,6000,27.25
3,2017-2018 Regular Season,10/17/2017,Jaylen Brown,Boston,Cleveland,R,39.60,29.352989,SG/SF,3700,36
4,2017-2018 Regular Season,10/17/2017,Kyrie Irving,Boston,Cleveland,R,39.35,21.846904,PG,8300,50.5


In [71]:
print(boston.shape, boston_players.shape, DFS_boston_players.shape, boston_stats_DFS.shape, BOS.shape)

(101, 50) (1078, 25) (1078, 11) (1078, 29) (1078, 81)


In [72]:
print(boston.columns, boston_players.columns, DFS_boston_players.columns)

Index(['DATASET', 'VENUE', '1Q', '2Q', '3Q', '4Q', 'OT1', 'OT2', 'OT3', 'OT4',
       'F', 'MIN', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'OR', 'DR', 'TOT',
       'A', 'PF', 'ST', 'TO', 'TO TO', 'BL', 'PTS', 'POSS', 'PACE', 'OEFF',
       'DEFF', 'REST_DAYS', 'STARTING LINEUPS', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'MAIN REF', 'CREW', 'OPENING ODDS',
       'OPENING_SPREAD', 'OPENING_TOTAL', 'MOVEMENTS', 'CLOSING ODDS',
       'MONEYLINE', 'HALFTIME', 'BOX SCORE', 'ODDS', 'GAME#'],
      dtype='object') Index(['DATASET', 'DATE', 'PLAYER', 'POSITION', 'TEAM', 'OPPONENT',
       'ROAD_HOME', 'MINUTES', 'FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'OR',
       'DR', 'TOT', 'A', 'PF', 'ST', 'TO', 'BL', 'PTS', 'DT_Double',
       'DraftKings_PTS'],
      dtype='object') Index(['DATASET', 'DATE', 'PLAYER', 'TEAM', 'OPPONENT', 'ROAD_HOME', 'MINUTES',
       'USAGE', 'DK_POS', 'DK_SAL', 'DK_PTS'],
      dtype='object')


In [73]:
#merge the individual player stats DF with the DFS stats DF
boston_stats_DFS = pd.merge(boston_players,DFS_boston_players,how='left')

In [74]:
boston_stats_DFS.head()

,DATASET,DATE,PLAYER,POSITION,TEAM,OPPONENT,ROAD_HOME,MINUTES,FG,FGA,...,ST,TO,BL,PTS,DT_Double,DraftKings_PTS,USAGE,DK_POS,DK_SAL,DK_PTS
0,2017-2018 Regular Season,10/17/2017,Gordon Hayward,SF,Boston,Cleveland,R,5.25,1,2,...,0,0,0,2,0.0,3.25,16.473616,SF,6700,3.25
1,2017-2018 Regular Season,10/17/2017,Jayson Tatum,SF,Boston,Cleveland,R,36.53,5,12,...,0,1,0,14,1.0,32.50,16.951635,SF/PF,3900,32.5
2,2017-2018 Regular Season,10/17/2017,Al Horford,C,Boston,Cleveland,R,32.12,2,7,...,0,0,1,9,0.0,27.25,13.570731,C,6000,27.25
3,2017-2018 Regular Season,10/17/2017,Jaylen Brown,SF,Boston,Cleveland,R,39.60,11,23,...,2,3,0,25,0.0,36.00,29.352989,SG/SF,3700,36
4,2017-2018 Regular Season,10/17/2017,Kyrie Irving,PG,Boston,Cleveland,R,39.35,8,17,...,3,2,0,22,1.0,50.50,21.846904,PG,8300,50.5


In [75]:
#joins all of our DFs and gives the team stats to each invidual
BOS = boston_stats_DFS.join(boston, on=['DATE', 'TEAM'],lsuffix='_player',rsuffix='_team')



In [76]:
BOS.columns

Index(['DATASET_player', 'DATE', 'PLAYER', 'POSITION', 'TEAM', 'OPPONENT',
       'ROAD_HOME', 'MINUTES', 'FG_player', 'FGA_player', '3P_player',
       '3PA_player', 'FT_player', 'FTA_player', 'OR_player', 'DR_player',
       'TOT_player', 'A_player', 'PF_player', 'ST_player', 'TO_player',
       'BL_player', 'PTS_player', 'DT_Double', 'DraftKings_PTS', 'USAGE',
       'DK_POS', 'DK_SAL', 'DK_PTS', 'DATASET_team', 'VENUE', '1Q', '2Q', '3Q',
       '4Q', 'OT1', 'OT2', 'OT3', 'OT4', 'F', 'MIN', 'FG_team', 'FGA_team',
       '3P_team', '3PA_team', 'FT_team', 'FTA_team', 'OR_team', 'DR_team',
       'TOT_team', 'A_team', 'PF_team', 'ST_team', 'TO_team', 'TO TO',
       'BL_team', 'PTS_team', 'POSS', 'PACE', 'OEFF', 'DEFF', 'REST_DAYS',
       'STARTING LINEUPS', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'MAIN REF', 'CREW', 'OPENING ODDS', 'OPENING_SPREAD',
       'OPENING_TOTAL', 'MOVEMENTS', 'CLOSING ODDS', 'MONEYLINE', 'HALFTIME',
       'BOX SCORE', 'ODDS', 'GA

In [77]:
BOS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1078 entries, 0 to 1077
Data columns (total 79 columns):
DATASET_player      1078 non-null object
DATE                1078 non-null object
PLAYER              1078 non-null object
POSITION            1078 non-null object
TEAM                1078 non-null object
OPPONENT            1078 non-null object
ROAD_HOME           1078 non-null object
MINUTES             1078 non-null float64
FG_player           1078 non-null int64
FGA_player          1078 non-null int64
3P_player           1078 non-null int64
3PA_player          1078 non-null int64
FT_player           1078 non-null int64
FTA_player          1078 non-null int64
OR_player           1078 non-null int64
DR_player           1078 non-null int64
TOT_player          1078 non-null int64
A_player            1078 non-null int64
PF_player           1078 non-null int64
ST_player           1078 non-null int64
TO_player           1078 non-null int64
BL_player           1078 non-null int64
PTS_

In [78]:
#creates colums that is a string of all the starters names
BOS['Starters'] = BOS[BOS.columns[62:67]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
#Creates a column 0 or 1 that signifies if the player was a starter for that game
BOS['Start'] = pd.Series()
for i,x in enumerate(BOS['PLAYER']):
    BOS['Start'][i] = BOS['PLAYER'][i] in BOS['Starters'][i]

/Users/willhall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [86]:
BOS.head()

,DATASET_player,DATE,PLAYER,POSITION,TEAM,OPPONENT,ROAD_HOME,MINUTES,FG_player,FGA_player,...,HALFTIME,BOX SCORE,ODDS,GAME#,Starters,Start,SPREAD,Favorite,Spread10,DKPTS_PER_MIN
0,2017-2018 Regular Season,10/17/2017,Gordon Hayward,SF,Boston,Cleveland,R,5.25,1,2,...,110,NaN,NaN,1,"Gordon Hayward,Jayson Tatum,Al Horford,Jaylen ...",1.0,4.0,True,False,0.619048
1,2017-2018 Regular Season,10/17/2017,Jayson Tatum,SF,Boston,Cleveland,R,36.53,5,12,...,110,NaN,NaN,1,"Gordon Hayward,Jayson Tatum,Al Horford,Jaylen ...",1.0,4.0,True,False,0.889680
2,2017-2018 Regular Season,10/17/2017,Al Horford,C,Boston,Cleveland,R,32.12,2,7,...,110,NaN,NaN,1,"Gordon Hayward,Jayson Tatum,Al Horford,Jaylen ...",1.0,4.0,True,False,0.848381
3,2017-2018 Regular Season,10/17/2017,Jaylen Brown,SF,Boston,Cleveland,R,39.60,11,23,...,110,NaN,NaN,1,"Gordon Hayward,Jayson Tatum,Al Horford,Jaylen ...",1.0,4.0,True,False,0.909091
4,2017-2018 Regular Season,10/17/2017,Kyrie Irving,PG,Boston,Cleveland,R,39.35,8,17,...,110,NaN,NaN,1,"Gordon Hayward,Jayson Tatum,Al Horford,Jaylen ...",1.0,4.0,True,False,1.283355


In [87]:
#column that is absolute value of spread
BOS['SPREAD'] = abs(BOS['OPENING_SPREAD'])
#Column singnifying if they are favorites
BOS['Favorite'] = BOS['OPENING_SPREAD'] >= 0
#Column signifying a spread greater than 10
BOS['Spread10'] = BOS['OPENING_SPREAD'] >= 10
#Column signifying fantasy points scored per minute
BOS['DKPTS_PER_MIN'] = BOS['DraftKings_PTS'] / BOS['MINUTES']

In [88]:
BOS.to_pickle('BOS.pkl')